In [1]:
from tinybig.config import rpn_config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'imdb_configs'
config_obj = rpn_config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'mps', 'random_seed': 1234, 'data_configs': {'data_class': 'tinybig.data.imdb', 'data_parameters': {'name': 'imdb', 'train_batch_size': 64, 'test_batch_size': 64}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 20, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.0001, 'weight_decay': 1e-05}}, 'lr_scheduler_configs': {'lr_scheduler_class': 'torch.optim.lr_scheduler.ExponentialLR', 'lr_scheduler_parameters': {'gamma': 0.9}}, 'loss_configs': {'loss_class': 'torch.nn.CrossEntropyLoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'model_parameters': {'name': 'reconciled_polynomial_network', 'depth': 3, 'depth_alloc': [1, 1, 1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_layer', 'layer_parameters': {'name': 'rpn_layer', 'm': 26964, 'n

In [2]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  mps ; random_seed:  1234


In [3]:
print('instantiating objects from config...')
object_dict = config_obj.instantiate_object_from_config(config['configurations'])
data_obj, model_obj, learner_obj, metric_obj, result_obj = [object_dict[name] for name in ['data', 'model', 'learner', 'metric', 'result']]
print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))

instantiating objects from config...
parameter num:  54572


In [4]:
print('loading dataset...')
data_loader = data_obj.load()

loading dataset...


In [5]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device, metric=metric_obj)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...


100%|██████████| 391/391 [00:05<00:00, 74.30it/s, epoch=0/20, loss=0.672, lr=0.0001, metric_score=0.85, time=5.3]  


Epoch: 0, Test Loss: 0.6756334919149004, Test Score: 0.82976, Time Cost: 2.7346367835998535


100%|██████████| 391/391 [00:04<00:00, 90.30it/s, epoch=1/20, loss=0.409, lr=9e-5, metric_score=0.825, time=12.4]


Epoch: 1, Test Loss: 0.4088795690432839, Test Score: 0.8542, Time Cost: 1.9856078624725342


...

Epoch: 17, Test Loss: 0.36797206425834494, Test Score: 0.88112, Time Cost: 1.9210257530212402


100%|██████████| 391/391 [00:04<00:00, 90.39it/s, epoch=18/20, loss=0.0401, lr=1.5e-5, metric_score=1, time=117]    


Epoch: 18, Test Loss: 0.3735352991067845, Test Score: 0.88036, Time Cost: 1.902376413345337


100%|██████████| 391/391 [00:04<00:00, 92.86it/s, epoch=19/20, loss=0.0422, lr=1.35e-5, metric_score=1, time=124]    


Epoch: 19, Test Loss: 0.378957481573686, Test Score: 0.88008, Time Cost: 1.875330924987793
model checkpoint saving to ./ckpt/imdb_configs_checkpoint...


In [6]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

testing model...


In [7]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating result...
accuracy 0.88008


In [8]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))

saving result...
